<a href="https://colab.research.google.com/github/andersonbraz/coursera-ibm-data-science-capstone/blob/master/Pandas_Exercise_Foursquare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Step 01 - Get Datas

In [1]:
import pandas as pd
import requests
import folium
from IPython.display import display

In [2]:
url_csv = 'https://raw.githubusercontent.com/andersonbraz/coursera-ibm-data-science-capstone/master/miami_dataset.csv'
miami_data = pd.read_csv(url_csv)
miami_data.head(99)

,Neighborhood,Latitude,Longitude
0,Allapattah,25.815,-80.224
1,Arts & Entertainment District,-25.799,-80.190
2,Brickell,25.758,-80.193
3,Buena Vista,25.813,-80.192
4,Coconut Grove,25.712,-80.257
5,Coral Way,25.750,-80.283
6,Design District,25.813,-80.193
7,Downtown,25.774,-80.193
8,Edgewater,25.802,-80.190
9,Flagami,25.762,-80.316


# Step 02 - Requisites API Foursquare

In [3]:
# PARAMETERS - API FOURSQUARE

CLIENT_ID = 'FW4LQTK3S5SUAAOCARVEFH4SC3PKQKCZRBPXT02DBRIPVMOT'
CLIENT_SECRET = 'SJG0NCWF02DOSDXMD0GE5LOOH4BFV110JKVIIKO4SFMDK4EK'
VERSION = '20180605'

# PARAMETERS - FOLIUM

LATITUDE_MIAMI = 25.7825453
LONGITUDE_MIAMI = -80.2994988

# Step 03 - Userful Functions

In [4]:
def get_venues(lat,lng):
    
    #set variables
    radius = 3000
    LIMIT = 100
    
    #url to fetch data from foursquare api
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
    
    # get all the data
    results = requests.get(url).json()
    venue_data = results["response"]['groups'][0]['items']
    venue_details = []
    for row in venue_data:
        try:
            venue_id = row['venue']['id']
            venue_name = row['venue']['name']
            venue_category = row['venue']['categories'][0]['name']
            venue_details.append([venue_id, venue_name, venue_category])
        except KeyError:
            pass
        
    column_names=['ID', 'Name', 'Category']
    df = pd.DataFrame(venue_details, columns = column_names)
    print("done!")
    return df

# Step 04 - Handler Data

In [5]:
target = 'Brazilian Restaurant'
columns_names = ['Neighborhood', 'ID', 'Name']
brazilian_restaurants_miami = pd.DataFrame(columns = columns_names)
count=1

for row in miami_data.values.tolist():
  Neighborhood, Latitude, Longitude = row
  venues = get_venues(Latitude, Longitude)
  brazilian_restaurants = venues[venues['Category'] == target]
  print('(',count,'/',len(miami_data),')', target + ' in ' + Neighborhood + ': ' + str(len(brazilian_restaurants)))

  for detail in brazilian_restaurants.values.tolist():
    id, name, category = detail
    brazilian_restaurants_miami = brazilian_restaurants_miami.append({'Neighborhood':Neighborhood, 'ID':id, 'Name':name, 'Category':category}, ignore_index = True)
  count+=1

done!
( 1 / 23 ) Brazilian Restaurant in Allapattah: 0
done!
( 2 / 23 ) Brazilian Restaurant in Arts & Entertainment District: 0
done!
( 3 / 23 ) Brazilian Restaurant in Brickell: 0
done!
( 4 / 23 ) Brazilian Restaurant in Buena Vista: 0
done!
( 5 / 23 ) Brazilian Restaurant in Coconut Grove: 0
done!
( 6 / 23 ) Brazilian Restaurant in Coral Way: 0
done!
( 7 / 23 ) Brazilian Restaurant in Design District: 0
done!
( 8 / 23 ) Brazilian Restaurant in Downtown: 1
done!
( 9 / 23 ) Brazilian Restaurant in Edgewater: 0
done!
( 10 / 23 ) Brazilian Restaurant in Flagami: 0
done!
( 11 / 23 ) Brazilian Restaurant in Grapeland Heights: 0
done!
( 12 / 23 ) Brazilian Restaurant in Liberty City: 0
done!
( 13 / 23 ) Brazilian Restaurant in Little Haiti: 0
done!
( 14 / 23 ) Brazilian Restaurant in Little Havana: 1
done!
( 15 / 23 ) Brazilian Restaurant in Lummus Park: 1
done!
( 16 / 23 ) Brazilian Restaurant in Midtown: 0
done!
( 17 / 23 ) Brazilian Restaurant in Overtown: 1
done!
( 18 / 23 ) Brazilian 

In [6]:
brazilian_restaurants_miami.head(99)

,Neighborhood,ID,Name,Category
0,Downtown,514b7bb8e4b0476ac82afd55,Steak Brasil,Brazilian Restaurant
1,Little Havana,514b7bb8e4b0476ac82afd55,Steak Brasil,Brazilian Restaurant
2,Lummus Park,514b7bb8e4b0476ac82afd55,Steak Brasil,Brazilian Restaurant
3,Overtown,514b7bb8e4b0476ac82afd55,Steak Brasil,Brazilian Restaurant
4,Park West,514b7bb8e4b0476ac82afd55,Steak Brasil,Brazilian Restaurant
5,Upper Eastside,4ae2706ef964a5204d8e21e3,Boteco Miami,Brazilian Restaurant


In [7]:
MIAMI_COORDINATES = (LATITUDE_MIAMI, LONGITUDE_MIAMI)
map_miami = folium.Map(location = MIAMI_COORDINATES, zoom_start=12)
display(map_miami)